In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
from sklearn import linear_model 
from sklearn import model_selection 
from sklearn import metrics
from sklearn import datasets

In [3]:
data = datasets.load_breast_cancer()
x = data.data
y = data.target

Napisati funkciju koja deli zadate skupove `x` i `y` na *trening* i *test* skupove u zadatoj razmeri.

Funkcija vraca skupove *x_train*, *x_test*, *y_train* i *y_test*.

Funkcija treba da oponasa funkcionalnosti `train_test_split()` metode.

In [4]:
def data_split(x, y, ratio):
    N = int(y.size * ratio)
    
    x_train = x[:N, :]
    y_train = y[:N]
    
    x_test = x[N:, :]
    y_test = y[N:]
    
    return x_train, x_test, y_train, y_test

In [5]:
x_train, x_test, y_train, y_test = data_split(x, y, 0.67)

In [6]:
y_train.shape

(381,)

In [7]:
y_test.shape

(188,)

Napisati funkciju koja uci zadati model na **trening** skupu,
a potom evaluira i vraca vrednost funkcije greske na 
**test** skupu. 

Funkcija treba da objedini funkcionalnosti `.fit()`, `.predict()` i `.score()` metoda.


In [8]:
def train_test_eval(model, x_train, y_train, x_test, y_test, error_function):
    model.fit(x_train, y_train)
    y_predicted = model.predict(x_test)
    return error_function(y_test, y_predicted)

Napisati funkciju koja podelom skupova `x` i  `y` u zadatoj razmeri
na *trening* i *validacioni* skup odredjuje iz skupa
zadatih konfiguracija onu vrednost konfiguracionog parametra
za koju model daje najmanju gresku.

Funkcija vraca **najbolji** model.

In [9]:
def train_valid_select(x, y, ratio, error_function, configure_model, configs):
    x_train, x_validation, y_train, y_validation = data_split(x, y, ratio)
    
    errors = []
    for c in configs:
        model = configure_model(c)
        error = train_test_eval(model, x_train, y_train,\
                                x_validation, y_validation, error_function)
        errors.append(error)
        
    errors = np.array(errors)
    
    # Biramo konfiguraciju modela koja ima najmanju gresku
    c_best = configs[np.argmin(errors)]
    
    # Konfigurisemo model za tu konfiguraciju.
    model = configure_model(c_best)
    
    # Obucavamo model na celokupnim podacima
    model.fit(x, y)
    
    return model

Napisati funkciju koja deli skupove na trening, test i validacione u zadatim razmerama, zatim odredjuje 
najbolji model koriscenjem `train_valid_select()` funkcije i vraca ocenu njegove greske koriscenjem 
`train_test_eval()` funkcije.

In [10]:
def train_valid_test_eval(x, y, ratios, error_function, configure_model, configs):
    # Delimo podatke na podatke za obucavanje/validaciju i testiranje.
    x_train_validation, x_test, y_train_validation, y_test =\
        data_split(x, y, ratios[0] + ratios[1])

    r = ratios[0] / (ratios[0] + ratios[1])
    
    # Trazimo model sa najboljom konfiguracijom.
    model = train_valid_select(x_train_validation, y_train_validation,\
                               r, error_function, configure_model, configs)
    
    # Evaluiramo dobijeni model na zasebnom skupu podataka za testiranje.
    y_predicted = model.predict(x_test)
    return model, error_function(y_test, y_predicted)

Funkcija omotac kojom se moze konfigurisati logisticka regresija.

In [11]:
def configure_logistic_model(c):
    return linear_model.LogisticRegression(C=c)

Funkcija koja se moze koristiti za ocenu greske klasifikacije.

In [12]:
def classification_error(y_true, y_predicted):
    return 1 - metrics.accuracy_score(y_true, y_predicted)

Skup vrednosti koji se moze koristiti za konfiguraciju modela. 

In [13]:
configurations = [10**i for i in range(-5, 5)]

Trazimo najbolju konfiguraciju za model i vracamo model koji je obucen sa tom najboljom konfiguracijom.

Ipak ne znamo ovde kakva je preciznost modela koji ce vratiti funkcija (primer sluzi da prikaze koriscenje funkcije). Dalji koraci bi ovde bili da se `model` evaluira na skupu podataka koji do sada **nije vidjen niti koriscen**.

In [14]:
model = train_valid_select(x, y, 0.8, classification_error,\
                           configure_logistic_model, configurations)

Vrsimo podelu na skupove za trening, validaciju i testiranje. Rezultat rada funkcije je
evaluirana preciznost/greska modela kao i model obucen na uniji skupova za trening i validaciju.

In [15]:
model, error = train_valid_test_eval(x, y, [0.6, 0.2, 0.2], classification_error,\
                      configure_logistic_model, configurations)
print('Error: {:.4}'.format(error))
print('Accuracy: {:.4}'.format(1-error))

Error: 0.07018
Accuracy: 0.9298


## Unakrsna validacija i ugnezdjena unakrsna validacija

Napisati funkciju koja koriscenjem unakrsne validacija evaluira zadati model. 

Funkcija treba da vrati vrednost greske modela.

In [16]:
def cross_validation_evaluation(model, x, y, number_of_folds, error_function):
    y_predicted = np.empty(y.size)
    
    # Generisemo indekse za odabir slojeva
    ix = np.arange(0, x.shape[0]) % number_of_folds
    
    for i in range(number_of_folds):
        x_train = x[ix != i, :]
        y_train = y[ix != i]
        
        y_test = y[ix == i]
        x_test = x[ix == i, :]
        
        model.fit(x_train, y_train)
        y_predicted[ix == i] = model.predict(x_test)
        
    return error_function(y, y_predicted)

#### Odabir najbolje konfiguracije koriscenjem unakrsne validacije

Napisati funkciju koja koriscenjem unakrsne validacije bira najbolju konfiguraciju za zadati model.

Funkcija vraca najbolji model. 

In [17]:
def cross_validation_selection(x, y, number_of_folds, error_function, configure_model, configs):
    errors = []
    
    for c in configs:
        model = configure_model(c)
        error = cross_validation_evaluation(model, x, y, number_of_folds, error_function)
        errors.append(error)
        
    errors = np.array(errors)
    c_best = configs[np.argmin(errors)]
    
    model = configure_model(c_best)
    model.fit(x, y)
    
    return model

Uocite slicnost sa funkcijom `train_valid_select`.

#### Ugnezdjena unakrsna validacija
Napisati funkciju koja koriscenjem ugnjezdene unakrsne velidacije daje ocenu greske modela.

In [18]:
def nested_cross_validation_evaluation(x, y, number_of_folds, error_function, configure_model, configs):
    y_predicted = np.empty(y.size)
    ix = np.arange(0, x.shape[0]) % number_of_folds
    
    for i in range(number_of_folds):
        x_train_validation = x[ix != i]
        y_train_validation = y[ix != i]
        
        x_test = x[ix == i]
        y_test = y[ix == i]
        
        model = cross_validation_selection(x_train_validation, y_train_validation,\
                                           number_of_folds, error_function, configure_model, configs)
        
        y_predicted[ix == i] = model.predict(x_test)
        
    return error_function(y, y_predicted)

In [19]:
nested_cross_validation_evaluation(x, y, 5, classification_error, configure_logistic_model, configurations)

0.04393673110720564